In [1]:
import numpy as np

In [2]:
from functions import get_data , calculate_sma



In [3]:
using_data=get_data('GOLD')

using_data=calculate_sma(using_data,[3,7,5,8,10,30,15,12,40,35])



In [4]:
import cProfile

In [5]:
# def find_prev_main(data, window, index):
#     df = data
#     high_points = df['High'].iloc[max(0, index - window):index + 1]
#     low_points = df['Low'].iloc[max(0, index - window):index + 1]
#     support = low_points.nsmallest(3).values.tolist()
#     resistance = high_points.nlargest(3).values.tolist()
#     return resistance, support

In [6]:
def find_prev_main3(data, window, index):
    df = data

    # Convert 'High' and 'Low' columns to NumPy arrays for slicing
    high_points = df['High'].values[max(0, index - window):index + 1]
    low_points = df['Low'].values[max(0, index - window):index + 1]

    # Vectorized operation to find smallest and largest values
    support = np.partition(low_points, 3)[:3].tolist()
    resistance = np.partition(high_points, -3)[-3:].tolist()

    return resistance, support

In [ ]:
def find_tp(trade_type, resistance, support, price):
    first_high = resistance[0]
    second_high = resistance[1]
    third_high = resistance[2]
    first_low = support[0]
    second_low = support[1]
    third_low = support[2]
    
    level1 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': 0, 'tp_price': 0, 'expected_profit': 0, 'expected_loss': 0}
    level2 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': 0, 'tp_price': 0, 'expected_profit': 0, 'expected_loss': 0}
    level3 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': 0, 'tp_price': 0, 'expected_profit': 0, 'expected_loss': 0}
    
    if trade_type == 'Buy':
        first_degree_low = price - first_low
        first_degree_high = first_high - price
        
        if first_degree_high < 0:
            level1 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': first_low, 'tp_price': 'invalid', 'expected_profit': first_degree_high, 'expected_loss': first_degree_low}
        else:
            ratio = first_degree_high / first_degree_low
            level1 = {'trade_type': trade_type, 'ratio': ratio, 'sl_price': first_low, 'tp_price': first_high, 'expected_profit': first_degree_high, 'expected_loss': first_degree_low}
        
        second_degree_low = price - second_low
        second_degree_high = second_high - price
        
        if second_degree_high < 0:
            level2 = level1
        else:
            ratio2 = second_degree_high / second_degree_low
            level2 = {'trade_type': trade_type, 'ratio': ratio2, 'sl_price': second_low, 'tp_price': second_high, 'expected_profit': second_degree_high, 'expected_loss': second_degree_low}
       
        third_degree_low = price - third_low
        third_degree_high = third_high - price
        
        if third_degree_high < 0:
            level3 = level2
        else:
            ratio3 = third_degree_high / third_degree_low
            level3 = {'trade_type': trade_type, 'ratio': ratio3, 'sl_price': third_low, 'tp_price': third_high, 'expected_profit': third_degree_high, 'expected_loss': third_degree_low}
        
    else:
        first_degree_low_sell = first_low - price
        first_degree_high_sell = price - first_low
        
        if first_degree_high_sell < 0: 
            level1 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': first_low, 'tp_price': 'invalid', 'expected_profit': first_degree_high_sell, 'expected_loss': first_degree_low_sell}
        else:
            ratio = first_degree_high_sell / first_degree_low_sell
            level1 = {'trade_type': trade_type, 'ratio': ratio, 'sl_price': first_low, 'tp_price': first_high, 'expected_profit': first_degree_high_sell, 'expected_loss': first_degree_low_sell}
        
        second_degree_low_sell = second_low - price
        second_degree_high_sell = price - second_low
        
        if second_degree_high_sell < 0:
            level2 = level1
        else:
            ratio2 = second_degree_high_sell / second_degree_low_sell
            level2 = {'trade_type': trade_type, 'ratio': ratio2, 'sl_price': second_low, 'tp_price': second_high, 'expected_profit': second_degree_high_sell, 'expected_loss': second_degree_low_sell}
        
        third_degree_low_sell = third_high - price
        third_degree_high_sell = price - third_low
        
        if third_degree_high_sell < 0:
            level3 = level2
        else:
            ratio3 = third_degree_high_sell / third_degree_low_sell
            level3 = {'trade_type': trade_type, 'ratio': ratio3, 'sl_price': third_low, 'tp_price': third_high, 'expected_profit': third_degree_high_sell, 'expected_loss': third_degree_low_sell}
        
    return level1, level2, level3


In [7]:
def find_tp2(trade_type, resistance, support, price):
    first_high, second_high, third_high = resistance[:3]
    first_low, second_low, third_low = support[:3]

    levels = np.zeros((3, 6), dtype=object)
    

    if trade_type == 'Buy':
        degree_low = price - np.array([first_low, second_low, third_low])
        degree_high = np.array([first_high, second_high, third_high]) - price

        invalid_mask = degree_high < 0

        levels[:, 0] = trade_type
        levels[:, 1] = 0
        levels[:, 2] = degree_low
        levels[:, 3] = np.where(invalid_mask, np.nan, degree_high)
        levels[:, 4] = np.where(invalid_mask, np.nan, degree_high)
        levels[:, 5] = np.where(invalid_mask, np.nan, degree_high)

        ratio_mask = ~invalid_mask
        non_zero_mask = levels[ratio_mask, 5] != 0

        valid_indices = ratio_mask & non_zero_mask
        levels_slice = levels[valid_indices, :]
        levels_slice[:, 1] = levels_slice[:, 3] / levels_slice[:, 5]

        levels[valid_indices, 1] = levels_slice[:, 1]
        levels[ratio_mask & ~non_zero_mask, 1] = np.nan

    else:
        degree_low_sell = np.array([first_low, second_low, third_low]) - price
        degree_high_sell = price - np.array([first_low, second_low, third_low])

        invalid_mask_sell = degree_high_sell < 0

        levels[:, 0] = trade_type
        levels[:, 1] = 0
        levels[:, 2] = degree_low_sell
        levels[:, 3] = np.where(invalid_mask_sell, np.nan, degree_high_sell)
        levels[:, 4] = np.where(invalid_mask_sell, np.nan, degree_low_sell)
        levels[:, 5] = np.where(invalid_mask_sell, np.nan, degree_high_sell)

        ratio_mask_sell = ~invalid_mask_sell
        non_zero_mask_sell = levels[ratio_mask_sell, 5] != 0

        valid_indices_sell = ratio_mask_sell & non_zero_mask_sell
        levels_slice_sell = levels[valid_indices_sell, :]
        levels_slice_sell[:, 1] = levels_slice_sell[:, 3] / levels_slice_sell[:, 5]

        levels[valid_indices_sell, 1] = levels_slice_sell[:, 1]
        levels[ratio_mask_sell & ~non_zero_mask_sell, 1] = np.nan

    level1 = {
        'trade_type': levels[0, 0],
        'ratio': levels[0, 1],
        'sl_price': levels[0, 2],
        'tp_price': levels[0, 3],
        'expected_profit': levels[0, 4],
        'expected_loss': levels[0, 5]
    }

    level2 = {
        'trade_type': levels[1, 0],
        'ratio': levels[1, 1],
        'sl_price': levels[1, 2],
        'tp_price': levels[1, 3],
        'expected_profit': levels[1, 4],
        'expected_loss': levels[1, 5]
    }

    level3 = {
        'trade_type': levels[2, 0],
        'ratio': levels[2, 1],
        'sl_price': levels[2, 2],
        'tp_price': levels[2, 3],
        'expected_profit': levels[2, 4],
        'expected_loss': levels[2, 5]
    }

    return level1, level2, level3


In [8]:
def mfind_tp(trade_type, resistance, support, price):
    first_high = resistance[0]
    second_high = resistance[1]
    third_high = resistance[2]
    first_low = support[0]
    second_low = support[1]
    third_low = support[2]
    
    level1 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': 0, 'tp_price': 0, 'expected_profit': 0, 'expected_loss': 0}
    level2 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': 0, 'tp_price': 0, 'expected_profit': 0, 'expected_loss': 0}
    level3 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': 0, 'tp_price': 0, 'expected_profit': 0, 'expected_loss': 0}
    
    if trade_type == 'Buy':
        first_degree_low = price - first_low
        first_degree_high = first_high - price
        
        if first_degree_high < 0:
            level1 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': first_low, 'tp_price': np.nan, 'expected_profit': first_degree_high, 'expected_loss': first_degree_low}
        else:
            ratio = first_degree_high / first_degree_low
            level1 = {'trade_type': trade_type, 'ratio': ratio, 'sl_price': first_low, 'tp_price': first_high, 'expected_profit': first_degree_high, 'expected_loss': first_degree_low}
        
        second_degree_low = price - second_low
        second_degree_high = second_high - price
        
        if second_degree_high < 0:
            level2 = level1
        else:
            ratio2 = second_degree_high / second_degree_low
            level2 = {'trade_type': trade_type, 'ratio': ratio2, 'sl_price': second_low, 'tp_price': second_high, 'expected_profit': second_degree_high, 'expected_loss': second_degree_low}
       
        third_degree_low = price - third_low
        third_degree_high = third_high - price
        
        if third_degree_high < 0:
            level3 = level2
        else:
            ratio3 = third_degree_high / third_degree_low
            level3 = {'trade_type': trade_type, 'ratio': ratio3, 'sl_price': third_low, 'tp_price': third_high, 'expected_profit': third_degree_high, 'expected_loss': third_degree_low}
        
    else:
        first_degree_low_sell = first_low - price
        first_degree_high_sell = price - first_low
        
        if first_degree_high_sell < 0: 
            level1 = {'trade_type': trade_type, 'ratio': 0, 'sl_price': first_low, 'tp_price': 'invalid', 'expected_profit': first_degree_high_sell, 'expected_loss': first_degree_low_sell}
        else:
            ratio = first_degree_high_sell / first_degree_low_sell
            level1 = {'trade_type': trade_type, 'ratio': ratio, 'sl_price': first_low, 'tp_price': first_high, 'expected_profit': first_degree_high_sell, 'expected_loss': first_degree_low_sell}
        
        second_degree_low_sell = second_low - price
        second_degree_high_sell = price - second_low
        
        if second_degree_high_sell < 0:
            level2 = level1
        else:
            ratio2 = second_degree_high_sell / second_degree_low_sell
            level2 = {'trade_type': trade_type, 'ratio': ratio2, 'sl_price': second_low, 'tp_price': second_high, 'expected_profit': second_degree_high_sell, 'expected_loss': second_degree_low_sell}
        
        third_degree_low_sell = third_high - price
        third_degree_high_sell = price - third_low
        
        if third_degree_high_sell < 0:
            level3 = level2
        else:
            ratio3 = third_degree_high_sell / third_degree_low_sell
            level3 = {'trade_type': trade_type, 'ratio': ratio3, 'sl_price': third_low, 'tp_price': third_high, 'expected_profit': third_degree_high_sell, 'expected_loss': third_degree_low_sell}
        
    return level1, level2, level3


In [ ]:
def find_tp2(trade_type, resistance, support, price):
    first_high, second_high, third_high = resistance[:3]
    first_low, second_low, third_low = support[:3]

    levels = np.zeros((3, 6), dtype=object)

    if trade_type == 'Buy':
        degree_low = price - np.array([first_low, second_low, third_low])
        degree_high = np.array([first_high, second_high, third_high]) - price

        invalid_mask = degree_high < 0

        levels[:, 0] = trade_type
        levels[:, 1] = 0
        levels[:, 2] = degree_low
        levels[:, 3] = np.where(invalid_mask, np.nan, degree_high)
        levels[:, 4] = np.where(invalid_mask, degree_high, degree_low)
        levels[:, 5] = np.where(invalid_mask, degree_low, degree_high)

        ratio_mask = ~invalid_mask
        non_zero_mask = levels[ratio_mask, 5] != 0

        valid_indices = ratio_mask & non_zero_mask
        levels_slice = levels[valid_indices, :]
        levels_slice[:, 1] = levels_slice[:, 3] / levels_slice[:, 5]

        levels[valid_indices, 1] = levels_slice[:, 1]
        levels[ratio_mask & ~non_zero_mask, 1] = np.nan

    else:
        degree_low_sell = np.array([first_low, second_low, third_low]) - price
        degree_high_sell = price - np.array([first_low, second_low, third_low])

        invalid_mask_sell = degree_high_sell < 0

        levels[:, 0] = trade_type
        levels[:, 1] = 0
        levels[:, 2] = degree_low_sell
        levels[:, 3] = np.where(invalid_mask_sell, np.nan, degree_high_sell)
        levels[:, 4] = np.where(invalid_mask_sell, degree_high_sell, degree_low_sell)
        levels[:, 5] = np.where(invalid_mask_sell, degree_low_sell, degree_high_sell)

        ratio_mask_sell = ~invalid_mask_sell
        non_zero_mask_sell = levels[ratio_mask_sell, 5] != 0

        valid_indices_sell = ratio_mask_sell & non_zero_mask_sell
        levels_slice_sell = levels[valid_indices_sell, :]
        levels_slice_sell[:, 1] = levels_slice_sell[:, 3] / levels_slice_sell[:, 5]

        levels[valid_indices_sell, 1] = levels_slice_sell[:, 1]
        levels[ratio_mask_sell & ~non_zero_mask_sell, 1] = np.nan

    level1 = {
        'trade_type': levels[0, 0],
        'ratio': levels[0, 1],
        'sl_price': levels[0, 2],
        'tp_price': levels[0, 3],
        'expected_profit': levels[0, 4],
        'expected_loss': levels[0, 5]
    }

    level2 = {
        'trade_type': levels[1, 0],
        'ratio': levels[1, 1],
        'sl_price': levels[1, 2],
        'tp_price': levels[1, 3],
        'expected_profit': levels[1, 4],
        'expected_loss': levels[1, 5]
    }

    level3 = {
        'trade_type': levels[2, 0],
        'ratio': levels[2, 1],
        'sl_price': levels[2, 2],
        'tp_price': levels[2, 3],
        'expected_profit': levels[2, 4],
        'expected_loss': levels[2, 5]
    }

    return level1, level2, level3


In [9]:
from functions import calc_prof

In [10]:
import time

In [11]:
def smastrategy(data, window,o_sma_value,s_sma_value):
    df = data
    trade = {}
    istrade = None
    istrade2=None
    istrade3=None
    
    openprice = 0
    tp1 = 0
    tp2 = 0
    tp3 = 0
    sl1 = 0
    sl2 = 0
    sl3 = 0
    start_time = time.time()
    opendate = 0
    processed_trades1 = []
    processed_trades2 = []
    processed_trades3 = []
    trade_class = None

    for i in range(window, len(df)):  # Adjusted the loop to start from the specified index
        if df[f'sma_{o_sma_value}'][i] > df[f'sma_{s_sma_value}'][i]:
            if istrade is None:
                trade_type = 'Buy'
                entryprice = df['Close'][i]
                resistance, support = find_prev_main3(data, window, i)
                t1, t2, t3 = mfind_tp(trade_type, resistance, support, entryprice)
                tp1 = t1['tp_price']
                tp2 = t2['tp_price']
                tp3 = t3['tp_price']
                sl1 = t1['sl_price']
                sl2 = t2['sl_price']
                sl3 = t3['sl_price']

                opendate = df['time'][i]
                istrade = 1
                istrade2=1
                istrade3=1
                if tp1 == 'invalid':
                    continue
                if tp1 == np.nan:
                    continue

                openprice = entryprice
                trade_class = trade_type
                
        elif df[f'sma_{o_sma_value}'][i] < df[f'sma_{s_sma_value}'][i]:
            if istrade is None:
                trade_type = 'Sell'
                trade_class = trade_type
                entryprice = df['Close'][i]
                opendate = df['time'][i]
                resistance, support = find_prev_main3(data, window, i)
                t1, t2, t3 = mfind_tp(trade_type, resistance, support, entryprice)
                openprice = entryprice
                tp1 = t1['tp_price']
                tp2 = t2['tp_price']
                tp3 = t3['tp_price']
                if tp1 == 'invalid':
                    continue
                if tp1 == np.nan:
                    continue
                istrade = 1
                istrade2=1
                istrade3=1
        
        price = data['Close'][i]
        if istrade==1:
            if trade_class == 'Buy':
                if price > tp1 or price < sl1:
                    istrade = None
                    closedate = df['time'][i]
                    profit = calc_prof(price, openprice)
                    processed_trades1.append(('Buy', opendate, closedate, profit))
            if trade_class == 'Sell':
                if price < tp1 or price > sl1:
                    istrade = None
                    closedate = df['time'][i]
                    profit = calc_prof(openprice, price)
                    processed_trades1.append(('Sell', opendate, closedate, profit))
        if istrade2==1:
            if trade_class == 'Buy':
                if price > tp2 or price < sl2:
                    istrade2 = None
                    closedate = df['time'][i]
                    profit = calc_prof(price, openprice)
                    processed_trades2.append(('Buy', opendate, closedate, profit))
            if trade_class == 'Sell':
                if price < tp2 or price > sl2:
                    istrade2 = None
                    closedate = df['time'][i]
                    profit = calc_prof(openprice, price)
                    processed_trades2.append(('Sell', opendate, closedate, profit))
        if istrade3==1:
            if trade_class == 'Buy':
                if price > tp3 or price < sl3:
                    istrade3 = None
                    closedate = df['time'][i]
                    profit = calc_prof(price, openprice)
                    processed_trades3.append(('Buy', opendate, closedate, profit))
            if trade_class == 'Sell':
                if price < tp3 or price > sl3:
                    istrade3 = None
                    closedate = df['time'][i]
                    profit = calc_prof(openprice, price)
                    processed_trades3.append(('Sell', opendate, closedate, profit))
        
    processed_trades=[processed_trades1,processed_trades2,processed_trades3]
    smastrategy_time = time.time() - start_time
    print(f"smastrategy execution time: {smastrategy_time} seconds")            

    return processed_trades


In [12]:
def smastrategy2(data, window, o_sma_value, s_sma_value):
    df = data
    trade = {}
    istrade = None
    istrade2 = None
    istrade3 = None

    openprice = 0
    tp1 = 0
    tp2 = 0
    tp3 = 0
    sl1 = 0
    sl2 = 0
    sl3 = 0
    start_time = time.time()
    opendate = 0
    processed_trades1 = []
    processed_trades2 = []
    processed_trades3 = []
    trade_class = None

    # Vectorized calculation of sma values
    sma_o_values = df[f'sma_{o_sma_value}'].values
    sma_s_values = df[f'sma_{s_sma_value}'].values

    for i in range(window, len(df)):
        if sma_o_values[i] > sma_s_values[i]:
            if istrade is None:
                trade_type = 'Buy'
                entryprice = df['Close'][i]
                resistance, support = find_prev_main3(data, window, i)
                t1, t2, t3 = mfind_tp(trade_type, resistance, support, entryprice)
                tp1 = t1['tp_price']
                tp2 = t2['tp_price']
                tp3 = t3['tp_price']
                sl1 = t1['sl_price']
                sl2 = t2['sl_price']
                sl3 = t3['sl_price']

                opendate = df['time'][i]
                istrade = 1
                istrade2 = 1
                istrade3 = 1
                if tp1 == 'invalid':
                    continue
                if tp1 == np.nan:
                    continue
                openprice = entryprice
                trade_class = trade_type
                
        elif sma_o_values[i] < sma_s_values[i]:
            if istrade is None:
                trade_type = 'Sell'
                trade_class = trade_type
                entryprice = df['Close'][i]
                opendate = df['time'][i]
                resistance, support = find_prev_main3(data, window, i)
                t1, t2, t3 = mfind_tp(trade_type, resistance, support, entryprice)
                openprice = entryprice
                tp1 = t1['tp_price']
                tp2 = t2['tp_price']
                tp3 = t3['tp_price']
                if tp1 == 'invalid':
                    continue
                if tp1 == np.nan:
                    continue
                istrade = 1
                istrade2 = 1
                istrade3 = 1
        
        price = data['Close'][i]
        if istrade == 1:
            if trade_class == 'Buy':
                if price > tp1 or price < sl1:
                    istrade = None
                    closedate = df['time'][i]
                    profit = calc_prof(price, openprice)
                    processed_trades1.append(('Buy', opendate, closedate, profit))
            if trade_class == 'Sell':
                if price < tp1 or price > sl1:
                    istrade = None
                    closedate = df['time'][i]
                    profit = calc_prof(openprice, price)
                    processed_trades1.append(('Sell', opendate, closedate, profit))
        if istrade2 == 1:
            if trade_class == 'Buy':
                if price > tp2 or price < sl2:
                    istrade2 = None
                    closedate = df['time'][i]
                    profit = calc_prof(price, openprice)
                    processed_trades2.append(('Buy', opendate, closedate, profit))
            if trade_class == 'Sell':
                if price < tp2 or price > sl2:
                    istrade2 = None
                    closedate = df['time'][i]
                    profit = calc_prof(openprice, price)
                    processed_trades2.append(('Sell', opendate, closedate, profit))
        if istrade3 == 1:
            if trade_class == 'Buy':
                if price > tp3 or price < sl3:
                    istrade3 = None
                    closedate = df['time'][i]
                    profit = calc_prof(price, openprice)
                    processed_trades3.append(('Buy', opendate, closedate, profit))
            if trade_class == 'Sell':
                if price < tp3 or price > sl3:
                    istrade3 = None
                    closedate = df['time'][i]
                    profit = calc_prof(openprice, price)
                    processed_trades3.append(('Sell', opendate, closedate, profit))
        
    processed_trades = [processed_trades1, processed_trades2, processed_trades3]         

    return processed_trades


In [13]:
from functions import evaluate

In [14]:
using_data=calculate_sma(using_data,[3,7,5,8,10,18,21,25,28,30,15,33,38,12,40,35])

In [15]:
list_sma=[3,7,5,8,10,18,21,25,28,30,15,33,38,12,40,35]
window_list=[5,10,15,20,25,30,35,40,45,50]

In [16]:
smatrials=list_sma
window=window_list

In [17]:
combinations = [(small_sma, large_sma, win) for small_sma in smatrials for large_sma in smatrials if small_sma != large_sma and small_sma < large_sma for win in window]

In [18]:
def results_(using_data,combinations):
    trades=[]
    evals1=[]
    evals2=[]
    evals3=[]
    for small_sma,large_sma, win in combinations:
        dfs = smastrategy2(using_data, win, small_sma, large_sma)
        trades.append((small_sma, large_sma, win, dfs))
        data_tp1=dfs[0]
        data_tp2=dfs[1]
        data_tp3=dfs[2]
        name=f'sma crossover {small_sma} & {large_sma} window{win}'
        evaluated_info=evaluate(data_tp1,name)
        evaluated_info2=evaluate(data_tp2,name)
        evaluated_info3=evaluate(data_tp3,name)
        result_dict1 = {
        'Strategy': name,
        'Take_profit_level':"level1(tp1)",
        'Max_Profit': evaluated_info[0],
        'Num_Buy_Trades': evaluated_info[1],
        'Total_Buy_Profit': evaluated_info[2],
        'Num_Sell_Trades': evaluated_info[3],
        'Total_Sell_Profit': evaluated_info[4],
        'Max_Profit_Index': evaluated_info[5],
        'Index_of_Max_Profit': evaluated_info[6],
        'Max_Duration': evaluated_info[7],
        'ID_Max_Duration': evaluated_info[8],
        'total_profitable_trades':evaluated_info[9],
        'total_unprofitable_trades':evaluated_info[10],
        'total_profitable_trades':evaluated_info[11],
        'total_loss_trades':evaluated_info[12]}
        
        evals1.append(result_dict1)
        result_dict2 = {
        'Strategy': name,
        'Take_profit_level':"level2(tp2)",
        'Max_Profit': evaluated_info2[0],
        'Num_Buy_Trades': evaluated_info2[1],
        'Total_Buy_Profit': evaluated_info2[2],
        'Num_Sell_Trades': evaluated_info2[3],
        'Total_Sell_Profit': evaluated_info2[4],
        'Max_Profit_Index': evaluated_info2[5],
        'Index_of_Max_Profit': evaluated_info2[6],
        'Max_Duration': evaluated_info2[7],
        'ID_Max_Duration': evaluated_info2[8],
        'total_profitable_trades':evaluated_info2[9],
        'total_unprofitable_trades':evaluated_info2[10],
        'total_profitable_trades':evaluated_info2[11],
        'total_loss_trades':evaluated_info2[12]}
        evals2.append(result_dict2)
        result_dict3 = {
        'Strategy': name,
        'Take_profit_level':"level3(tp3)",
        'Max_Profit': evaluated_info3[0],
        'Num_Buy_Trades': evaluated_info3[1],
        'Total_Buy_Profit': evaluated_info3[2],
        'Num_Sell_Trades': evaluated_info3[3],
        'Total_Sell_Profit': evaluated_info3[4],
        'Max_Profit_Index': evaluated_info3[5],
        'Index_of_Max_Profit': evaluated_info3[6],
        'Max_Duration': evaluated_info3[7],
        'ID_Max_Duration': evaluated_info3[8],
        'total_profitable_trades':evaluated_info3[9],
        'total_unprofitable_trades':evaluated_info3[10],
        'total_profitable_trades':evaluated_info3[11],
        'total_loss_trades':evaluated_info3[12]}
        
        evals3.append(result_dict3)
    retuning_values=[trades,evals1,evals2,evals3] 
    return tuple(retuning_values)        


In [19]:
finished_evaluation=results_(using_data,combinations)

C:\Users\DELL\AppData\Local\Temp\ipykernel_13228\3360572759.py:48: RuntimeWarning: invalid value encountered in scalar divide
  ratio = first_degree_high_sell / first_degree_low_sell
C:\Users\DELL\AppData\Local\Temp\ipykernel_13228\3360572759.py:57: RuntimeWarning: invalid value encountered in scalar divide
  ratio2 = second_degree_high_sell / second_degree_low_sell
C:\Users\DELL\AppData\Local\Temp\ipykernel_13228\3360572759.py:48: RuntimeWarning: invalid value encountered in scalar divide
  ratio = first_degree_high_sell / first_degree_low_sell
C:\Users\DELL\AppData\Local\Temp\ipykernel_13228\3360572759.py:57: RuntimeWarning: invalid value encountered in scalar divide
  ratio2 = second_degree_high_sell / second_degree_low_sell
C:\Users\DELL\AppData\Local\Temp\ipykernel_13228\3360572759.py:57: RuntimeWarning: invalid value encountered in scalar divide
  ratio2 = second_degree_high_sell / second_degree_low_sell
C:\Users\DELL\AppData\Local\Temp\ipykernel_13228\3360572759.py:48: RuntimeWa

In [20]:
import pandas as pd

In [21]:
trades, evals1, evals2, evals3 = finished_evaluation

# Convert the lists of dictionaries to pandas DataFrames
df_evals1 = pd.DataFrame(evals1)
df_evals2 = pd.DataFrame(evals2)
df_evals3 = pd.DataFrame(evals3)

# Write the DataFrames to CSV files
df_evals1.to_csv('evals1.csv', index=False)
df_evals2.to_csv('evals2.csv', index=False)
df_evals3.to_csv('evals3.csv', index=False)

# Create a DataFrame for the "trades" information
df_trades = pd.DataFrame(trades, columns=['small_sma', 'large_sma', 'win', 'dfs'])

# Write the "trades" DataFrame to a CSV file
df_trades.to_csv('trades.csv', index=False)